# ME314 Final Project

### Submission instructions

- Include all of your code (and handwritten solutions when applicable) used to complete the problems.
- Highlight your answers (i.e. **bold** and outline the answers) for handwritten or markdown questions and include simplified code outputs (e.g. .simplify()) for python questions.
- Enable Google Colab permission for viewing 
 * Click Share in the upper right corner
 * Under "Get Link" click "Share with..." or "Change" 
 * Then make sure it says "Anyone with Link" and "Editor" under the dropdown menu
- Make sure all cells are run before submitting (i.e. check the permission by running your code in a private mode)
 * Please don't make changes to your file after submitting, so we can grade it!
- Submit a link to your Google Colab file that has been run (before the submission deadline) and don't edit it afterwards!

In [1]:
##############################################################################################
# If you're using Google Colab, uncomment this section by selecting the whole section and press
# ctrl+'/' on your and keyboard. Run it before you start programming, this will enable the nice 
# LaTeX "display()" function for you. If you're using the local Jupyter environment, leave it alone
##############################################################################################
import sympy as sym
import numpy as np

# def custom_latex_printer(exp,**options):
#     from google.colab.output._publish import javascript
#     url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/3.1.1/latest.js?config=TeX-AMS_HTML"
#     javascript(url=url)
#     return sym.printing.latex(exp,**options)
# sym.init_printing(use_latex="mathjax",latex_printer=custom_latex_printer)

In [2]:
#helper functions

def SOnAndRnToSEn(R, p):
           
    #do type checking for the matrix types
    if type(R) == list:
        R = np.matrix(R)
        
    n = R.shape[0]
    if ((R.shape[0] != R.shape[1]) or                               #R is NP array or Sym matrix
        ((type(p) is np.ndarray and max(p.shape) != R.shape[0]) or  #p is NP array and shape mismatch or.. 
          ((isinstance(p, list) or isinstance(p, sym.Matrix)) and 
            ( len(p) != R.shape[0] ))   )  ):                       #p is Sym matrix or "list" and shape mismatch
        raise Exception(f"Shape of R {R.shape} and p ({len(p)}) mismatch; exiting.")
        return None
        
    #construct a matrix based on returning a Sympy Matrix
    if isinstance(R, sym.Matrix) or isinstance(p, sym.Matrix): 
        #realistically one of these needs to be symbolic to do this

        if isinstance(R, np.ndarray) or isinstance(p, np.ndarray):
            raise Exception("R and p cannot mix/match Sympy and Numpy types")
            return None
        
        G = sym.zeros(n+1)
        G[:n, n] = sym.Matrix(p)
    
    #construct a matrix based on returning a Numpy matrix
    elif isinstance(R, np.ndarray) or isinstance(R, list):
        G = np.zeros([n+1, n+1])
        # print(f"\nSOnAndRnToSEn Debug: \n\nR:\n{R}    \n\np:\n{p}   ")
        G[:n, n] = np.array(p).T
        
    else:
        raise Exception("Error: type not recognized")
        return None
    
    G[:n,:n] = R
    G[-1,-1] = 1
    return G  

def SEnToSOnAndRn(SEnmat):
    '''Decomposes a SE(n) vector into its rotation matrix and displacement components.
    '''
    if isinstance(SEnmat, list):
        SEnmat = np.matrix(SEnmat)
    n = SEnmat.shape[0]
    return SEnmat[:(n-1), :(n-1)], SEnmat[:(n-1), n-1]

#test cases

#SO(2) and R2 - numpy
mat1 = np.matrix([[1,2],[3,4]])
p1 = [5,6]
out = SOnAndRnToSEn(mat1, p1)
assert np.array_equal(out,  np.matrix([[1,2,5],[3,4,6],[0,0,1]]) ), f"{out}"

#SO(2) and R2 - sympy
mat2 = sym.Matrix([[5,6],[7,8]])
p2 = [9,0]
out = SOnAndRnToSEn(mat2, p2)
assert out - sym.Matrix([[5,6,9],[7,8,0],[0,0,1]]) == sym.zeros(3,3), f"{out}"

#SO(3) and R3 - numpy 
mat3 = np.matrix([[1,2,3],[4,5,6],[7,8,9]])
p3 = [1.1,2.2,3.3]
out = SOnAndRnToSEn(mat3, p3)
assert np.array_equal(out,  np.matrix([[1,2,3,1.1],[4,5,6,2.2],[7,8,9,3.3],[0,0,0,1]]) ), f"{out}"

#SO(3) and R3 - sympy 
mat4 = sym.Matrix([[1,2,3],[4,5,6],[7,8,9]])
p4 = [4.4,5.5,6.6]
out = SOnAndRnToSEn(mat4, p4)
diff = out - sym.Matrix([[1,2,3,4.4],[4,5,6,5.5],[7,8,9,6.6],[0,0,0,1]])
assert diff == sym.zeros(4,4), f"{out}\n\n{diff}"

#dimensional mismatch - check that it throws an error
#SOnAndRnToSEn(mat2, p4)

#type mismatch - check that it throws an error
#SOnAndRnToSEn(mat2, sym.Matrix(p1))
#SOnAndRnToSEn(mat1, np.matrix(p2))

#SE(3)
SE3mat = SOnAndRnToSEn(np.identity(3), [1,2,3])
[SO3, R3] = SEnToSOnAndRn(SE3mat)
assert np.array_equal(SO3, np.identity(3)) and np.array_equal(R3, [1,2,3]), f"{SO3}\n{R3}"

#SE(2)
SE3mat = SOnAndRnToSEn(np.identity(2), [4,5])
[SO2, R2] = SEnToSOnAndRn(SE3mat)
assert np.array_equal(SO2, np.identity(2)) and np.array_equal(R2, [4,5]), f"{SO2}\n{R2}"

print("All assertions passed")


All assertions passed


In [3]:
def HatVector3(w):
    '''Turns a vector in R3 to a skew-symmetric matrix in so(3). 
    Works with both Sympy and Numpy matrices.
    '''   
    #create different datatype representations based on type of w
    if isinstance(w, list) or isinstance(w, np.ndarray) \
        or isinstance(w, np.matrix):
        f = np.array 
    elif isinstance(w, sym.Matrix): #NP and Sym
        f = sym.Matrix

    return f([
        [    0, -w[2],  w[1]],
        [ w[2],     0, -w[0]],
        [-w[1],  w[0],     0]
    ])

###

def UnhatMatrix3(w_hat):
    '''Turns a skew-symmetric matrix in so(3) into a vector in R3.
    '''
    if isinstance(w_hat, list) or isinstance(w_hat, np.ndarray) \
        or isinstance(w_hat, np.matrix):
        f = np.array
        w_hat = np.array(w_hat)
    elif isinstance(w_hat, sym.Matrix) or isinstance(w_hat, sym.ImmutableMatrix):
        f = sym.Matrix
    else:
        raise Exception(f"UnhatMatrix3: Unexpected type of w_hat: {type(w_hat)}")
    
    #matrix checking, for use in potential debug. generalized to both Sympy and Numpy
    same = np.array([w_hat + w_hat.T == f([
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]
    ]
    ) ] )
    
    if (not same.all()):
        raise Exception("UnhatMatrix3: w_hat not skew_symmetric")
    
    #NP and Sym
    return f([
        -w_hat[1,2],
        w_hat[0,2],
        -w_hat[0,1],
    ])

def InvSEn(SEnmat):
    '''Takes the inverse of a SE(n) matrix.
    Compatible with Numpy, Sympy, and list formats.
    '''
    if isinstance(SEnmat, list):
        SEnmat = np.matrix(SEnmat)
    ###
    n = SEnmat.shape[0]
    R = SEnmat[:(n-1), :(n-1)]
    p = SEnmat[:(n-1),   n-1 ]
        
    return SOnAndRnToSEn(R.T, -R.T @ p)
    
###

#testing
t = sym.symbols(r't')
theta1 = sym.Function(r'\theta_1')(t)
theta2 = sym.Function(r'\theta_2')(t)

w1 = [6,5,4]

w_hat1 = [
    [0, -9, 8],
    [9, 0, -7],
    [-8, 7, 0]
]

w_hat2 = [
    [10, -9, 8],
    [9, 0, -7],
    [-8, 7, 0]
]

w_hat3 = sym.Matrix([
    [0, -theta1, 8],
    [theta1, 0, -7],
    [-8, 7, 0]
])

T1 = [
    [1,  2,  3,  4],
    [5,  6,  7,  8],
    [9, 10, 11, 12],
    [0,  0,  0,  1]
]

print(f"\nHat: \n {w1} \n{HatVector3(w1)}")
print(f"\nUnhat: \n{w_hat1} \n{UnhatMatrix3(w_hat1)}")
#print(f"\nNon-Skew-symm unhat: \n{w_hat2} \n{UnhatMatrix3(w_hat2)}")
print(f"\nSymbolic unhat: \n{w_hat3} \n{UnhatMatrix3(w_hat3)}")

print(f"\nTransInv: \n{T1} \n{InvSEn(T1)}")


Hat: 
 [6, 5, 4] 
[[ 0 -4  5]
 [ 4  0 -6]
 [-5  6  0]]

Unhat: 
[[0, -9, 8], [9, 0, -7], [-8, 7, 0]] 
[7 8 9]

Symbolic unhat: 
Matrix([[0, -\theta_1(t), 8], [\theta_1(t), 0, -7], [-8, 7, 0]]) 
Matrix([[7], [8], [\theta_1(t)]])

TransInv: 
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [0, 0, 0, 1]] 
[[   1.    5.    9. -152.]
 [   2.    6.   10. -176.]
 [   3.    7.   11. -200.]
 [   0.    0.    0.    1.]]


In [4]:
def InertiaMatrix6(m, scriptI):
    '''Takes the mass and inertia matrix properties of an object in space,
    and constructs a 6x6 matrix corresponding to [[mI 0]; [0 scriptI]].
    Currently only written for Sympy matrix representations.
    '''
    if (m.is_Matrix or not scriptI.is_square):
        raise Exception("Type error: m or scriptI in InertiaMatrix6")
        
    mat = sym.zeros(6)
    mI = m * sym.eye(3)
    mat[:3, :3] = mI
    mat[3:6, 3:6] = scriptI
    return mat

def HatVector6(vec):
    '''Convert a 6-dimensional body velocity into a 4x4 "hatted" matrix,
    [[w_hat v]; [0 0]], where w_hat is skew-symmetric.
    w = 
    '''
    if isinstance(vec, np.matrix) or isinstance(vec, np.ndarray):
        vec = np.array(vec).flatten()
    
    v = vec[:3]
    w = vec[3:6]
    
    #this ensures if there are symbolic variables, they stay in Sympy form
    if isinstance(vec, sym.Matrix):
        v = sym.Matrix(v)
        w = sym.Matrix(w)
        
    w_hat = HatVector3(w)
    
    #note that the result isn't actually in SE(3) but 
    #that the function below creates a 4x4 matrix from a 3x3 and
    #1x3 matrix - with type checking - so we'll use it
    mat = SOnAndRnToSEn(w_hat, v)
    return mat

def UnhatMatrix4(mat):
    '''Convert a 4x4 "hatted" matrix,[[w_hat v]; [0 0]], into a 6-dimensional
    body velocity [v, w].
    '''
    #same as above - matrices aren't SE(3) and SO(3) but the function
    #can take in a 4x4 mat and return a 3x3 and 3x1 mat
    [w_hat, v] = SEnToSOnAndRn(mat)
    w = UnhatMatrix3(w_hat)
    
    if (isinstance(w, np.matrix) or isinstance(w, np.ndarray)):
        return np.array([v, w]).flatten()
    elif isinstance(w, sym.Matrix):
        return sym.Matrix([v, w])
    else:
        raise Exception("Unexpected datatype in UnhatMatrix4")
    
### testing
test1, test2, test3 = sym.symbols(r'test_1, test_2, test_3')
vec1 = np.matrix([1,2,3,4,5,6])
vec2 = sym.Matrix([test1, test2, test3, test1, test2, test3])
vec3 = np.array([1, 2, 3, 4, 5, 6])

#inertia matrix testing
#---------------------------#

#print("InertiaMatrix6 tests:")

##not currently configured to work
## m1 = 4
## scriptI1 = 7*np.eye(3)

m2 = sym.symbols(r'test_m')
scriptI2 = sym.symbols(r'test_J') * sym.eye(3)
## print(InertiaMatrix6(m1, scriptI1))
#display(InertiaMatrix6(m2, scriptI2))


#---------------------------#

mat1 = HatVector6(vec1)
mat2 = HatVector6(vec2)
mat3 = HatVector6(vec3)

#print("HatVector6 tests:")
# print(type(mat1))
# print(mat1, end='\n\n')

# print(type(mat2))
# display(mat2)

# print(type(mat3))
# print(mat3, end='\n\n')

#---------------------------#

vec4 = UnhatMatrix4(mat1)
vec5 = UnhatMatrix4(mat2)
vec6 = UnhatMatrix4(mat3)

# print("UnhatMatrix4 tests:")
# print(type(vec4))
# print(vec4, end='\n\n')

# print(type(vec5))
# display(vec5)

# print(type(vec6))
# print(vec6, end='\n\n')

In [5]:
def CalculateVb6(G):
    '''Calculate the body velocity, a 6D vector [v, w], given a trans-
    formation matrix G from one frame to another.
    '''
    G_inv = InvSEn(G)
    Gdot = G.diff(t) #for sympy matrices, this also carries out chain rule 
    V_hat = G_inv @ Gdot 
    
    if isinstance(G, sym.Matrix):
        V_hat = sym.simplify(V_hat)
        
    return UnhatMatrix4(V_hat)

#testing
t = sym.symbols(r't')
x = sym.Function(r'x')(t)
y = sym.Function(r'y')(t)
theta1 = sym.Function(r'\theta_1')(t)
theta2 = sym.Function(r'\theta_2')(t)

R = sym.Matrix([
    [sym.cos(-theta2), -sym.sin(-theta2), 0],
    [sym.sin(-theta2),  sym.cos(-theta2), 0],
    [              0,                0,   1]

])

G = SOnAndRnToSEn(R, [x,y,0])
V = CalculateVb6(G)
print("\nV:")
display(V)

#


V:


Matrix([
[-sin(\theta_2(t))*Derivative(y(t), t) + cos(\theta_2(t))*Derivative(x(t), t)],
[ sin(\theta_2(t))*Derivative(x(t), t) + cos(\theta_2(t))*Derivative(y(t), t)],
[                                                                           0],
[                                                                           0],
[                                                                           0],
[                                                 -Derivative(\theta_2(t), t)]])

In [6]:
def compute_EL_lhs(lagrangian, q):
    '''
    Helper function for computing the Euler-Lagrange equations for a given system,
    so I don't have to keep writing it out over and over again.
    
    Inputs:
    - lagrangian: our Lagrangian function in symbolic (Sympy) form
    - q: our state vector [x1, x2, ...], in symbolic (Sympy) form
    
    Outputs:
    - eqn: the Euler-Lagrange equations in Sympy form
    '''
    
    # wrap system states into one vector (in SymPy would be Matrix)
    #q = sym.Matrix([x1, x2])
    qd = q.diff(t)
    qdd = qd.diff(t)

    # compute derivative wrt a vector, method 1
    # wrap the expression into a SymPy Matrix
    L_mat = sym.Matrix([lagrangian])
    dL_dq = L_mat.jacobian(q)
    dL_dqdot = L_mat.jacobian(qd)

    #set up the Euler-Lagrange equations
    #LHS = dL_dq - dL_dqdot.diff(t)
    LHS = dL_dqdot.diff(t) - dL_dq
    
    return LHS.T

def solve_EL(eqn, var):
    '''
    Helper function to solve and display the solution for the Euler-Lagrange
    equations.
    
    Inputs:
    - eqn: Euler-Lagrange equation (type: Sympy Equation())
    - var: state vector (type: Sympy Matrix). typically a form of q-doubledot
        but may have different terms
    
    Outputs:
    - Prints symbolic solutions
    - Returns symbolic solutions in a dictionary
    '''
    
    #soln = sym.solve(eqn, var, dict = True)
    soln = sym.solve_poly_system(eqn, var)
    eqns_solved = []
    print("Solve_EL: solved.")
    
    for i, sol in enumerate(soln):
        for x in list(sol.keys()):
            eqn_solved = sym.Eq(x, sol[x])
            eqns_solved.append(eqn_solved)
            
    return eqns_solved

def solve_EL_hw7(lamb_list, phi_list, F_mat, q, lhs):
    """This version of the Euler-Lagrange equations uses both constraints 
    and external forcing terms.
    
    Inputs:
    - lamb_list: a list of symbolic variables lambda1, lambda2, ...
    - phi_list: a list of constraint equations phi1, phi2, ...
        - must be the same length as list of lambda variables
    - F_mat: a *Sympy Matrix* of forcing terms. Needs to be the same 
        dimension in the vertical direction as the LHS of the Euler-Lagrange EQs
    - lhs: the left-hand side of the Euler-Lagrange EQs, passed in from compute_EL()
    - q: the state vector, in Sympy Matrix form
    """
    
    qd = q.diff(t)
    qdd = qd.diff(t)
    expr_matrix = lhs - F_mat
    phidd_matrix = sym.Matrix([0])
    phidd_list = []
    
    #add constraint terms to matrix of expressions to solve
    for i in range(len(phi_list)):
        phi = phi_list[i]
        lamb = lamb_list[i]
        
        phidd = phi.diff(t).diff(t)
        lamb_grad = sym.Matrix([lamb * phi.diff(a) for a in q])
        qdd = qdd.row_insert(len(qdd), sym.Matrix([lamb]) )
    
        #format equations so they're all in one matrix
        expr_matrix = expr_matrix - lamb_grad
        phidd_list.append(sym.Matrix([phidd])  )

    for phidd_matrix in phidd_list:
        expr_matrix = expr_matrix.row_insert(len(expr_matrix), phidd_matrix)
        
    ###------FOR HW7 ONLY:--------#
    L = sym.symbols(r'L')
    theta1dd = theta1.diff(t).diff(t)
    expr_matrix = expr_matrix.subs({L1:L, L2:L})
#     expr_matrix = sym.collect(expr_matrix, theta1dd)
    
    print("Equations to be solved (LHS - lambda * grad(phi) = 0):")
    RHS = sym.zeros(len(expr_matrix), 1)
    disp_eq = sym.Eq(expr_matrix, RHS)
    display(disp_eq)
    print("Variables to solve for:")
    display(qdd)

    #solve E-L equations
    eqns_solved = solve_EL(expr_matrix, qdd)
    return eqns_solved


## Milestone 1

simulation runs with no damping, no user interaction; not real time,
	just initial conditions and simulation

In [29]:
#define frames and symbols. let L1 = L2, m1 = m2 for computational efficiency
#as this condition is unlikely to change
L, w, m, g = sym.symbols(r'L, w, m, g')
t = sym.symbols(r't')

x = sym.Function(r'x')(t)
y = sym.Function(r'y')(t)
theta1 = sym.Function(r'\theta_1')(t)
theta2 = sym.Function(r'\theta_2')(t)
phi1 = sym.Function(r'\Phi_1')(t)
phi2 = sym.Function(r'\Phi_2')(t)

q = sym.Matrix([x, y, theta1, theta2, phi1, phi2])
q_ext = sym.Matrix([        x,         y,         theta1,         theta2,         phi1,         phi2,
                    x.diff(t), y.diff(t), theta1.diff(t), theta2.diff(t), phi1.diff(t), phi2.diff(t)])

subs_dict = {
    L : 1,
    w : 1/3.0,
    m : 1,
    g : 9.81,
}

#---------------right side---------------#

Rab = sym.Matrix([
    [sym.cos(theta2), -sym.sin(theta2), 0],
    [sym.sin(theta2),  sym.cos(theta2), 0],
    [              0,                0, 1],
])

RdB2 = sym.Matrix([
    [sym.cos(phi2), -sym.sin(phi2), 0],
    [sym.sin(phi2),  sym.cos(phi2), 0],
    [            0,              0, 1],
])

p_bd = sym.Matrix([0, -L, 0])

Gab = SOnAndRnToSEn(Rab, [0, 0, 0])
Gbd = SOnAndRnToSEn(sym.eye(3), p_bd)
GdB2 = SOnAndRnToSEn(RdB2, [0, 0, 0])

Gsa = SOnAndRnToSEn(sym.eye(3), [x, y, 0])
Gsb = Gsa @ Gab
Gsd = Gsb @ Gbd
GsB2 = Gsd @ GdB2


#---------------left side---------------#

Rac = sym.Matrix([
    [sym.cos(theta1), -sym.sin(theta1), 0],
    [sym.sin(theta1),  sym.cos(theta1), 0],
    [              0,                0, 1],
])

ReB1 = sym.Matrix([
    [sym.cos(phi1), -sym.sin(phi1), 0],
    [sym.sin(phi1),  sym.cos(phi1), 0],
    [            0,              0, 1],
])

p_ce = sym.Matrix([0, -L, 0])

Gac = SOnAndRnToSEn(Rac, [0, 0, 0])
Gce = SOnAndRnToSEn(sym.eye(3), p_ce)
GeB1 = SOnAndRnToSEn(ReB1, [0, 0, 0])

Gsa = SOnAndRnToSEn(sym.eye(3), [x, y, 0])
Gsc = Gsa @ Gac
Gse = Gsc @ Gce
GsB1 = Gse @ GeB1


In [19]:
#define important positions      
ym1 = ( Gsg    @ sym.Matrix([0, 0, 0, 1]) )[0]
ym2 = ( Gsf    @ sym.Matrix([0, 0, 0, 1]) )[0]
posn_top = Gsa @ sym.Matrix([0, 0, 0, 1])

In [20]:
#define kinetic and potential energy

Vbf = CalculateVb6(Gsf)
Vbg = CalculateVb6(Gsg)

# inertia_f = m * w**2 * sym.eye(3)
# inertia_g = 

scriptI_f = m * sym.Matrix([
    [w**2,    0,      0],
    [   0, w**2,      0],
    [   0,    0, 2*w**2]
])

scriptI_g = m * sym.Matrix([
    [w**2,    0,      0],
    [   0, w**2,      0],
    [   0,    0, 2*w**2]
])

inertia_f = InertiaMatrix6(m, scriptI_f)
inertia_g = InertiaMatrix6(m, scriptI_g)

In [21]:
KE_f = 0.5 * (Vbf.T @ inertia_f @ Vbf)[0]
KE_g = 0.5 * (Vbg.T @ inertia_g @ Vbg)[0]

display(KE_f)
display(KE_g)

U = m*g*(ym1 + ym2)

lagrangian = KE_f + KE_g - U
lagrangian = lagrangian.simplify()

print("Lagrangian:")
display(lagrangian)

1.0*m*w**2*(Derivative(\Phi_2(t), t) + Derivative(\theta_2(t), t))**2 + 0.5*m*(-L*sin(\Phi_2(t))*Derivative(\theta_2(t), t) - sin(\Phi_2(t) + \theta_2(t))*Derivative(x(t), t) + cos(\Phi_2(t) + \theta_2(t))*Derivative(y(t), t))**2 + 0.5*m*(L*cos(\Phi_2(t))*Derivative(\theta_2(t), t) + sin(\Phi_2(t) + \theta_2(t))*Derivative(y(t), t) + cos(\Phi_2(t) + \theta_2(t))*Derivative(x(t), t))**2

1.0*m*w**2*(Derivative(\Phi_1(t), t) + Derivative(\theta_1(t), t))**2 + 0.5*m*(-L*sin(\Phi_1(t))*Derivative(\theta_1(t), t) - sin(\Phi_1(t) + \theta_1(t))*Derivative(x(t), t) + cos(\Phi_1(t) + \theta_1(t))*Derivative(y(t), t))**2 + 0.5*m*(L*cos(\Phi_1(t))*Derivative(\theta_1(t), t) + sin(\Phi_1(t) + \theta_1(t))*Derivative(y(t), t) + cos(\Phi_1(t) + \theta_1(t))*Derivative(x(t), t))**2

Lagrangian:


m*(0.5*L**2*Derivative(\theta_1(t), t)**2 + 0.5*L**2*Derivative(\theta_2(t), t)**2 - 1.0*L*g*sin(\theta_1(t)) - 1.0*L*g*sin(\theta_2(t)) + 1.0*L*sin(\theta_1(t))*Derivative(\theta_1(t), t)*Derivative(y(t), t) + 1.0*L*sin(\theta_2(t))*Derivative(\theta_2(t), t)*Derivative(y(t), t) + 1.0*L*cos(\theta_1(t))*Derivative(\theta_1(t), t)*Derivative(x(t), t) + 1.0*L*cos(\theta_2(t))*Derivative(\theta_2(t), t)*Derivative(x(t), t) - 2.0*g*x(t) + 1.0*w**2*Derivative(\Phi_1(t), t)**2 + 2.0*w**2*Derivative(\Phi_1(t), t)*Derivative(\theta_1(t), t) + 1.0*w**2*Derivative(\Phi_2(t), t)**2 + 2.0*w**2*Derivative(\Phi_2(t), t)*Derivative(\theta_2(t), t) + 1.0*w**2*Derivative(\theta_1(t), t)**2 + 1.0*w**2*Derivative(\theta_2(t), t)**2 + 1.0*Derivative(x(t), t)**2 + 1.0*Derivative(y(t), t)**2)

In [ ]:
def SpringForce(Gsa, Grs, Grg, Ggm):
    '''Calculates the force acting on the 
    '''
    
    
    #define transformation 
    
    
    
    #springposn minus posn of end effector, in b,  will just be springposn - [0 0 0]
    dp = pb[:]
    
    #direction of force given by unit vector of springposn
    unit_dir = -dp / np.linalg.norm(dp)
    
    #magnitude of force given by (stiffness) * (norm(springPosn_b) - restlength)
    mag_force = stiffness * (np.linalg.norm(pb) - restLength)
    sf = (mag_force * unit_dir).T[0].tolist()    

In [28]:
#impact conditions

#define positions of vertices
v1bar = w*sym.Matrix([ 1/sym.sqrt(2),  1/sym.sqrt(2), 0, 1])
v2bar = w*sym.Matrix([-1/sym.sqrt(2),  1/sym.sqrt(2), 0, 1])
v3bar = w*sym.Matrix([-1/sym.sqrt(2), -1/sym.sqrt(2), 0, 1])
v4bar = w*sym.Matrix([ 1/sym.sqrt(2), -1/sym.sqrt(2), 0, 1])

GB1B2 = InvSEn(GsB1) @ GsB2
GB2B1 = InvSEn(GB1B2)

#subscripts: let v12 mean "the 2nd vertex of the 1st body"\
#the home frame of v1n is in body 1; impact cond. requires posn in 2nd body frame
v11_B2 = sym.simplify(GB2B1 @ v1bar)
v12_B2 = sym.simplify(GB2B1 @ v2bar)
v13_B2 = sym.simplify(GB2B1 @ v3bar)
v14_B2 = sym.simplify(GB2B1 @ v4bar)

#size of blocks is the same, so posn of vertices v2n in B2 frame is same
#as posn of v1n in B1 frame
v21_B1 = sym.simplify(GB1B2 @ v1bar)
v22_B1 = sym.simplify(GB1B2 @ v2bar)
v23_B1 = sym.simplify(GB1B2 @ v3bar)
v24_B1 = sym.simplify(GB1B2 @ v4bar)

#find x and y components of posn
v11x_B2, v11y_B2 = v11_B2
v12x_B2, v12y_B2 = v12_B2
v13x_B2, v13y_B2 = v13_B2
v14x_B2, v14y_B2 = v14_B2

v21x_B1, v21y_B1 = v21_B1
v22x_B1, v22y_B1 = v22_B1
v23x_B1, v23y_B1 = v23_B1
v24x_B1, v24y_B1 = v24_B1
#---#

print("Sample expression of vertex x coordinate:")           
display(v21x_B1)


w*(L*sin(\Phi_1(t) + \theta_1(t))*cos(\theta_1(t)) - L*sin(\Phi_1(t) + \theta_1(t))*cos(\theta_2(t)) - L*sin(\theta_1(t))*cos(\Phi_1(t) + \theta_1(t)) + L*sin(\theta_2(t))*cos(\Phi_1(t) + \theta_1(t)) + sqrt(2)*sin(\Phi_1(t) - \Phi_2(t) + \theta_1(t) - \theta_2(t))/2 + sqrt(2)*cos(\Phi_1(t) - \Phi_2(t) + \theta_1(t) - \theta_2(t))/2)

In [31]:
#substitute in values 
v11x_B2_np = sym.lambdify(q, v11x_B2)
v11y_B2_np = sym.lambdify(q, v11y_B2)

v12x_B2_np = sym.lambdify(q, v12x_B2)
v12y_B2_np = sym.lambdify(q, v12y_B2)

v13x_B2_np = sym.lambdify(q, v13x_B2)
v13y_B2_np = sym.lambdify(q, v13y_B2)

v14x_B2_np = sym.lambdify(q, v14x_B2)
v14y_B2_np = sym.lambdify(q, v14y_B2)

###

v21x_B1_np = sym.lambdify(q, v21x_B1)
v21y_B1_np = sym.lambdify(q, v21y_B1)

v22x_B1_np = sym.lambdify(q, v22x_B1)
v22y_B1_np = sym.lambdify(q, v22y_B1)

v23x_B1_np = sym.lambdify(q, v23x_B1)
v23y_B1_np = sym.lambdify(q, v23y_B1)

v24x_B1_np = sym.lambdify(q, v24x_B1)
v24y_B1_np = sym.lambdify(q, v24y_B1)

In [33]:
#define full set of impact conditions
wval = subs_dict[w]
def impact_condition(s):
    '''Contains and evaluates an array of impact conditions for the current
    system, at the current state s. 
    
    Returns: a logical true/false as to whether any impact condition was met;
        list of indices of impact conditions that were met
    '''
    
    impact_conds = np.array([
        -wval/2.0 < v11x_B2_np(s) < wval/2.0   and   -wval/2.0 < v11y_B2_np(s) < wval/2.0,
        -wval/2.0 < v12x_B2_np(s) < wval/2.0   and   -wval/2.0 < v12y_B2_np(s) < wval/2.0,
        -wval/2.0 < v13x_B2_np(s) < wval/2.0   and   -wval/2.0 < v13y_B2_np(s) < wval/2.0,
        -wval/2.0 < v14x_B2_np(s) < wval/2.0   and   -wval/2.0 < v14y_B2_np(s) < wval/2.0,
        
        -wval/2.0 < v21x_B1_np(s) < wval/2.0   and   -wval/2.0 < v21y_B1_np(s) < wval/2.0,
        -wval/2.0 < v22x_B1_np(s) < wval/2.0   and   -wval/2.0 < v22y_B1_np(s) < wval/2.0,
        -wval/2.0 < v23x_B1_np(s) < wval/2.0   and   -wval/2.0 < v23y_B1_np(s) < wval/2.0,
        -wval/2.0 < v24x_B1_np(s) < wval/2.0   and   -wval/2.0 < v24y_B1_np(s) < wval/2.0,     
    ])
    
    #find any impact conditions that have been met
    impact_met = np.any(impact_conds)
    impact_indices = np.nonzero(impact_conds)[0].tolist() #indices where true
    
    return impact_met, impact_indices

In [ ]:
#evaluate different phi_list values numerically to see which have
#just occurred
def phi_nearzero(s, atol):
    '''Takes in the current system state s, and returns the indices of
    impact conditions phi(q) that are close to 0 at the given instant in
    time. 
    
    The results of this function will then be used to determine which of
    the symbolically solved impact updates will be applied to the system.
    
    Parameters:
    - s: current state of system: x, y, theta1, theta2, phi1, phi2.
    - atol: absolute tolerance for "close to zero", passed to np.isclose().
        Example: atol = 1E-4
    
    Returns:
    - a list of indices of phi that are near zero at the given time.
    '''
    f_list = np.array([
        v11x_B2_np, v11y_B2_np,
        v12x_B2_np, v12y_B2_np,
        v13x_B2_np, v13y_B2_np,
        v14x_B2_np, v14y_B2_np,

        v21x_B1_np, v21y_B1_np,
        v22x_B1_np, v22y_B1_np,
        v23x_B1_np, v23y_B1_np,
        v24x_B1_np, v24y_B1_np,
    ])
    
    #apply upper and lower bound condition to all vertices, in x and y directions
    phi_arr_np = np.array([])
    for i in np.arange(0, len(f_list)):
        phi_arr_np = np.append(phi_arr_np, f_list[i](s) + wval/2.0)
        phi_arr_np = np.append(phi_arr_np, f_list[i](s) - wval/2.0)

    #we're interested in which of the phi conditions were evaluated at close to 0, so return the
    #list of indices close to 0
    closetozero = np.isclose( phi_arr_np, np.zeros(phi_arr_np.shape), atol=atol )
    
    #this gives the indices of phi close to 0
    any_nearzero = np.any(closetozero)
    phi_indices = np.nonzero(closetozero)[0].tolist()
    
    return any_nearzero, phi_indices
    
def filter_phiq(impact_indices, phi_indices):
    '''Simultaneous impact must be considered for this project, as the 
    user interaction means initial conditions cannot be pre-set such that
    no simultaneous impacts occur. 
    
    In the case of simultaneous impact of 2 cubes of the same size, there are
    potential phi(q) for indices impacting walls that approach zero even when 
    no impact is occuring at those vertices. Ex: for an exact head-on collision
    of two blocks [ ][ ], the top left vertex of box 1 is "at" the vertical boundary
    of the second box, even though no impact is occurring.
    
    This function filters the indices of phi(q) that are near zero and returns only
    the indices of phi(q) near zero that correspond to impact conditions (evaluated in
    impact_indices) that have been satisfied.
    
    Args:
    - phi_indices    (NP array): passed from phi_nearzero()
    - impact_indices (NP array): passed from impact_condition()
    
    Returns:
    - valid_phiq     (NP array): a subset of phi_indices that corresponds to an 
                                    element of impact_indices
    '''
    inds = np.in1d(phi_indices//4, impact_indices) #evaluates whether elements of phi_indices
                                                    #are related to an impact condition, T/F
    c = np.array(np.nonzero(inds)[0]) #turns locations of these T values to indices
    valid_phiq = phi_indices[c]  
    return valid_phiq                 #returns the phi(q) equations that both evaluate to ~0 and
                                       #are related to an impact condition that has been met
    

In [12]:
#impact conditions and update


In [13]:
#helper functions for GUI operations
import tkinter as tk
import time
import numpy as np

def make_oval(canvas: tk.Canvas, center: tuple, width: int, height: int, fill: str='hotpink'):
    top_left = (center[0] - width, center[1] - height)
    bottom_right = (center[0] + width, center[1] + height)
    return canvas.create_oval([top_left, bottom_right], fill=fill, width=0) #return content ID

def make_circle(canvas: tk.Canvas, center: tuple, radius: int, fill: str='hotpink'):
    return make_oval(canvas, center, radius, radius, fill=fill) #return content ID

def make_grid_label(canvas, x, y, w, h, offset):
    coord = x if not x==0 else h-y
    
    canvas.create_oval(
        x - offset, 
        y - offset, 
        x + offset,  
        y + offset, 
        fill='black'
    )
    canvas.create_text(
        x + offset, 
        y + offset, 
        text="{0}".format(coord),
        anchor="sw", 
        font=("Purisa", 8)
    )

def make_grid(canvas, w, h):
    interval = 100

    # Delete old grid if it exists:
    canvas.delete('grid_line')
    # Creates all vertical lines at intervals of 100
    for i in range(0, w, interval):
        canvas.create_line(i, 0, i, h, tag='grid_line', fill='gray', dash=(2,2))

    # Creates all horizontal lines at intervals of 100
    for i in range(0, h, interval):
        canvas.create_line(0, i, w, i, tag='grid_line', fill='gray', dash=(2,2))

    # Creates axis labels
    offset = 2
    for y in range(0, h, interval):
        make_grid_label(canvas, 0, y, w, h, offset)
        
    for x in range(0, w, interval):
        make_grid_label(canvas, x, h, w, h, offset)
        
def make_coordsys(canvas, x, y, line_length, tag):
    canvas.create_line(x, y, x + line_length,               y, arrow=tk.LAST, tag=tag+'x')
    canvas.create_line(x, y,               x, y - line_length, arrow=tk.LAST, tag=tag+'y')



In [14]:
%matplotlib tk

In [15]:
root = tk.Tk()
root.title("Final Project")
height = 500
width = 600
canvas = tk.Canvas(root, bg="white", height=height, width=width)

line = canvas.create_line(0, 0, 400, 400)
coordsys_len = 50
user_coordsys = make_coordsys(canvas, 300, 400, coordsys_len, tag='user_pos')

rad = 5
pend_pos = make_circle(canvas, (150, 150), 2)
make_grid(canvas, width, height)

#define frames relative to GUI frame. not all of these will be 
#rigid body transformations belonging to SO(3) or SE(3) - flipping
#the y axis of the GUI is not SO(3)

Rgr = np.matrix([
    [1, 0, 0],
    [0, -1, 0],
    [0, 0, 1]
])

Ggr = SOnAndRnToSEn(Rgr, [0, 0, 0])
Grs = SOnAndRnToSEn(np.identity(3), [width/2, -height, 0])

def on_mouse_click(event):
    print(f"Mouse clicked at: {event.x} {event.y}")

def on_mouse_over(event):
    #canvas.coords(line, 0, 0, event.x, event.y)
    canvas.coords('user_posx', 
                 event.x, event.y,
                 event.x + coordsys_len, event.y
             )  
    
    canvas.coords('user_posy', 
                 event.x, event.y,
                 event.x, event.y - coordsys_len
             )
    
def on_key(event):
    print(event.char)
    print(event.keycode)
    if event.char.lower() == 'q':
        root.destroy()

def on_frame(event):
    pass
    
canvas.bind("<Button>", on_mouse_click)
canvas.bind("<Motion>", on_mouse_over)
canvas.bind("<Key>", on_key)


'2081547460160on_key'

Run the cells below to start the simulation.

In [16]:
canvas.pack()
root.mainloop()

## Milestone 2

simulation runs with damping, no user interaction

## Milestone 3

simulation runs with damping and a sinusoidal forcing applied to the
	base of the clackers

## Milestone 4

simulation runs with no user input and base of pendula following a 
	predetermined path using springForce() method

## Milestone 5

simulation runs in real time with mouse posn and springForce